In [ ]:
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown, HBox, VBox, Output, Layout, BoundedFloatText
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

# --- Enhanced Calculator Function ---
def enhanced_retirement_calculator(
    age_current,
    age_retire,
    life_expectancy,
    monthly_benefit_income,
    portfolio_total,
    yearly_investment,
    years_contributing,
    current_monthly_budget,
    inflation_rate,
    expected_return,
    account_type,
    tax_rate,
    retirement_budget_ratio,
    retirement_phases,
    healthcare_costs
):
    # Calculate years in retirement
    retirement_years = life_expectancy - age_retire
    years_until_retirement = age_retire - age_current

    # Ensure years_contributing doesn't exceed years_until_retirement
    years_contributing = min(years_contributing, years_until_retirement)

    # Create pre-retirement portfolio projection
    pre_retirement_df = pd.DataFrame(index=range(years_until_retirement + 1))
    pre_retirement_df.loc[0, 'Age'] = age_current
    pre_retirement_df.loc[0, 'Portfolio'] = portfolio_total
    pre_retirement_df.loc[0, 'Yearly_Contribution'] = yearly_investment if years_contributing > 0 else 0

    # Calculate portfolio growth during working years with contributions
    for year in range(1, years_until_retirement + 1):
        pre_retirement_df.loc[year, 'Age'] = age_current + year

        # Add contribution if within contributing years
        contribution = yearly_investment if year <= years_contributing else 0
        pre_retirement_df.loc[year, 'Yearly_Contribution'] = contribution

        # Calculate growth with contribution assumed to be made at start of year
        previous_portfolio = pre_retirement_df.loc[year-1, 'Portfolio']
        investment_return = (previous_portfolio + contribution) * expected_return
        pre_retirement_df.loc[year, 'Portfolio'] = previous_portfolio + contribution + investment_return

    # Adjust portfolio value for growth until retirement
    future_portfolio = pre_retirement_df.loc[years_until_retirement, 'Portfolio']

    # Create dataframe for year-by-year projections
    df = pd.DataFrame(index=range(retirement_years + 1))

    # Set up initial values
    df.loc[0, 'Age'] = age_retire
    df.loc[0, 'Portfolio'] = future_portfolio
    df.loc[0, 'Annual_Benefit'] = monthly_benefit_income * 12

    # Initial budget adjusted for inflation until retirement
    initial_retirement_budget = current_monthly_budget * 12 * retirement_budget_ratio * (1 + inflation_rate) ** years_until_retirement
    df.loc[0, 'Annual_Budget'] = initial_retirement_budget

    # Healthcare initial costs
    df.loc[0, 'Healthcare_Costs'] = healthcare_costs * 12 * (1 + inflation_rate) ** years_until_retirement

    # Apply different spending phases if selected
    phase_adjustments = {
        'Constant': [1.0] * retirement_years,
        'Early Active': [1.2] * min(10, retirement_years) + [0.9] * max(0, retirement_years - 10),
        'Late Increase': [0.9] * min(20, retirement_years) + [1.3] * max(0, retirement_years - 20)
    }

    spending_adjustments = phase_adjustments[retirement_phases]

    # Year-by-year calculations
    for year in range(1, retirement_years + 1):
        df.loc[year, 'Age'] = age_retire + year

        # Adjust benefit for inflation
        df.loc[year, 'Annual_Benefit'] = df.loc[year-1, 'Annual_Benefit'] * (1 + inflation_rate)

        # Adjust budget for inflation and phase of retirement
        phase_factor = spending_adjustments[year-1] if year-1 < len(spending_adjustments) else spending_adjustments[-1]
        df.loc[year, 'Annual_Budget'] = df.loc[0, 'Annual_Budget'] * (1 + inflation_rate) ** year * phase_factor

        # Healthcare costs grow faster than inflation
        df.loc[year, 'Healthcare_Costs'] = df.loc[year-1, 'Healthcare_Costs'] * (1 + inflation_rate + 0.02)

        # Calculate withdrawal needed
        total_expenses = df.loc[year, 'Annual_Budget'] + df.loc[year, 'Healthcare_Costs']

        # If using Roth (tax-free withdrawals), no tax adjustment needed
        if account_type == 'Roth IRA/401k':
            withdrawal_needed = total_expenses - df.loc[year, 'Annual_Benefit']
        else:
            # For traditional accounts, adjust for taxes
            withdrawal_needed = (total_expenses - df.loc[year, 'Annual_Benefit']) / (1 - tax_rate)

        df.loc[year, 'Withdrawal'] = max(0, withdrawal_needed)

        # Calculate withdrawal rate
        df.loc[year, 'Withdrawal_Rate'] = df.loc[year, 'Withdrawal'] / df.loc[year-1, 'Portfolio'] if df.loc[year-1, 'Portfolio'] > 0 else 0

        # Update portfolio value
        starting_portfolio = df.loc[year-1, 'Portfolio']
        investment_returns = (starting_portfolio - df.loc[year, 'Withdrawal'] / 2) * expected_return
        df.loc[year, 'Portfolio'] = starting_portfolio - df.loc[year, 'Withdrawal'] + investment_returns
        df.loc[year, 'Portfolio'] = max(0, df.loc[year, 'Portfolio'])

        # Track if we've run out of money
        if df.loc[year, 'Portfolio'] <= 0 and year < retirement_years:
            df.loc[year:, 'Portfolio'] = 0
            break

    # Monte Carlo simulation for success probability
    num_simulations = 1000
    success_count = 0

    historical_returns = np.random.normal(expected_return, 0.12, (num_simulations, retirement_years))

    for sim in range(num_simulations):
        sim_portfolio = future_portfolio
        for year in range(retirement_years):
            if sim_portfolio <= 0:
                break

            annual_benefit = monthly_benefit_income * 12 * (1 + inflation_rate) ** (year + years_until_retirement)

            # Apply spending phase adjustment
            phase_factor = spending_adjustments[year] if year < len(spending_adjustments) else spending_adjustments[-1]
            annual_budget = initial_retirement_budget * (1 + inflation_rate) ** year * phase_factor

            # Healthcare with additional growth
            healthcare_cost = healthcare_costs * 12 * (1 + inflation_rate + 0.02) ** (year + years_until_retirement)

            total_expenses = annual_budget + healthcare_cost

            # Apply tax treatment based on account type
            if account_type == 'Roth IRA/401k':
                withdrawal = max(0, total_expenses - annual_benefit)
            else:
                withdrawal = max(0, (total_expenses - annual_benefit) / (1 - tax_rate))

            # Apply this year's random return
            sim_portfolio = sim_portfolio - withdrawal + (sim_portfolio - withdrawal/2) * historical_returns[sim, year]

        if sim_portfolio > 0:
            success_count += 1

    success_probability = success_count / num_simulations * 100

    # Output calculation results
    output = Output()
    with output:
        print(f"\n--- Enhanced Retirement Projection ---")
        print(f"Current Age: {age_current}, Retirement Age: {age_retire}, Life Expectancy: {life_expectancy}")
        print(f"Years Until Retirement: {years_until_retirement}, Years In Retirement: {retirement_years}")
        print(f"\nInitial Monthly Benefits: ${monthly_benefit_income:,.2f}")
        print(f"Current Portfolio: ${portfolio_total:,.2f}")
        print(f"Yearly Investment: ${yearly_investment:,.2f} for {years_contributing} years")
        print(f"Projected Portfolio at Retirement: ${future_portfolio:,.2f}")
        print(f"Current Monthly Budget: ${current_monthly_budget:,.2f}")
        print(f"Retirement Budget Ratio: {retirement_budget_ratio:.0%}")
        print(f"Initial Retirement Monthly Budget: ${initial_retirement_budget/12:,.2f}")
        print(f"Monthly Healthcare Costs at Retirement: ${df.loc[0, 'Healthcare_Costs']/12:,.2f}")
        print(f"\nExpected Annual Return: {expected_return:.2%}")
        print(f"Inflation Rate: {inflation_rate:.2%}")
        print(f"Effective Tax Rate: {tax_rate:.0%}")
        print(f"Retirement Spending Pattern: {retirement_phases}")

        if df['Portfolio'].iloc[-1] > 0:
            print(f"\nRESULT: Portfolio survives to end of life expectancy")
            print(f"Final Portfolio Value at Age {life_expectancy}: ${df['Portfolio'].iloc[-1]:,.2f}")
        else:
            portfolio_depletion_age = df.loc[df['Portfolio'] <= 0, 'Age'].min()
            print(f"\nRESULT: Portfolio depleted at age {portfolio_depletion_age:.0f}")

        print(f"\nMonte Carlo Success Probability: {success_probability:.1f}%")

        # Create visualization
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15))

        # Pre-retirement portfolio growth
        ax1.plot(pre_retirement_df['Age'], pre_retirement_df['Portfolio'], 'g-', linewidth=2)
        ax1.bar(pre_retirement_df['Age'], pre_retirement_df['Yearly_Contribution'], color='blue', alpha=0.3, label='Annual Contributions')
        ax1.set_title('Pre-Retirement Portfolio Growth')
        ax1.set_xlabel('Age')
        ax1.set_ylabel('Portfolio Value ($)')
        ax1.grid(True)
        ax1.legend()

        # Portfolio value over time during retirement
        ax2.plot(df['Age'], df['Portfolio'], 'b-', linewidth=2)
        ax2.set_title('Retirement Portfolio Value by Age')
        ax2.set_xlabel('Age')
        ax2.set_ylabel('Portfolio Value ($)')
        ax2.grid(True)

        # Income, expenses, and withdrawals
        ax3.bar(df['Age'], df['Annual_Benefit'], label='Benefit Income', color='green', alpha=0.6)
        ax3.bar(df['Age'], df['Withdrawal'], bottom=df['Annual_Benefit'], label='Portfolio Withdrawals', color='red', alpha=0.6)
        ax3.plot(df['Age'], df['Annual_Budget'], 'k--', label='Living Expenses', linewidth=2)
        ax3.plot(df['Age'], df['Healthcare_Costs'], 'r--', label='Healthcare Costs', linewidth=2)
        ax3.plot(df['Age'], df['Annual_Budget'] + df['Healthcare_Costs'], 'b-', label='Total Expenses', linewidth=2)
        ax3.set_title('Retirement Income and Expenses by Age')
        ax3.set_xlabel('Age')
        ax3.set_ylabel('Amount ($)')
        ax3.grid(True)
        ax3.legend()

        plt.tight_layout()
        plt.show()

        # Display the pre-retirement data table
        print("\nPre-Retirement Portfolio Growth:")
        display_cols_pre = ['Age', 'Portfolio', 'Yearly_Contribution']
        display(pre_retirement_df[display_cols_pre].head().append(pre_retirement_df[display_cols_pre].tail()))
        display_cols = ['Age', 'Portfolio', 'Annual_Benefit', 'Annual_Budget', 'Healthcare_Costs', 'Withdrawal', 'Withdrawal_Rate']
        pd.set_option('display.float_format', '${:.2f}'.format)
        display(pd.concat([df[display_cols].head(5), df[display_cols].tail(5)]))

    return output

from ipywidgets import interact, FloatSlider, IntSlider, Dropdown, HBox, VBox, Output, Layout, BoundedFloatText
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

# --- Enhanced Calculator Function ---
def enhanced_retirement_calculator(
    age_current,
    age_retire,
    life_expectancy,
    monthly_benefit_income,
    portfolio_total,
    yearly_investment,
    years_contributing,
    current_monthly_budget,
    inflation_rate,
    expected_return,
    account_type,
    tax_rate,
    retirement_budget_ratio,
    retirement_phases,
    healthcare_costs
):
    # Calculate years in retirement
    retirement_years = life_expectancy - age_retire
    years_until_retirement = age_retire - age_current

    # Ensure years_contributing doesn't exceed years_until_retirement
    years_contributing = min(years_contributing, years_until_retirement)

    # Create pre-retirement portfolio projection
    pre_retirement_df = pd.DataFrame(index=range(years_until_retirement + 1))
    pre_retirement_df.loc[0, 'Age'] = age_current
    pre_retirement_df.loc[0, 'Portfolio'] = portfolio_total
    pre_retirement_df.loc[0, 'Yearly_Contribution'] = yearly_investment if years_contributing > 0 else 0

    # Calculate portfolio growth during working years with contributions
    for year in range(1, years_until_retirement + 1):
        pre_retirement_df.loc[year, 'Age'] = age_current + year

        # Add contribution if within contributing years
        contribution = yearly_investment if year <= years_contributing else 0
        pre_retirement_df.loc[year, 'Yearly_Contribution'] = contribution

        # Calculate growth with contribution assumed to be made at start of year
        previous_portfolio = pre_retirement_df.loc[year-1, 'Portfolio']
        investment_return = (previous_portfolio + contribution) * expected_return
        pre_retirement_df.loc[year, 'Portfolio'] = previous_portfolio + contribution + investment_return

    # Adjust portfolio value for growth until retirement
    future_portfolio = pre_retirement_df.loc[years_until_retirement, 'Portfolio']

    # Create dataframe for year-by-year projections
    df = pd.DataFrame(index=range(retirement_years + 1))

    # Set up initial values
    df.loc[0, 'Age'] = age_retire
    df.loc[0, 'Portfolio'] = future_portfolio
    df.loc[0, 'Annual_Benefit'] = monthly_benefit_income * 12

    # Initial budget adjusted for inflation until retirement
    initial_retirement_budget = current_monthly_budget * 12 * retirement_budget_ratio * (1 + inflation_rate) ** years_until_retirement
    df.loc[0, 'Annual_Budget'] = initial_retirement_budget

    # Healthcare initial costs
    df.loc[0, 'Healthcare_Costs'] = healthcare_costs * 12 * (1 + inflation_rate) ** years_until_retirement

    # Apply different spending phases if selected
    phase_adjustments = {
        'Constant': [1.0] * retirement_years,
        'Early Active': [1.2] * min(10, retirement_years) + [0.9] * max(0, retirement_years - 10),
        'Late Increase': [0.9] * min(20, retirement_years) + [1.3] * max(0, retirement_years - 20)
    }

    spending_adjustments = phase_adjustments[retirement_phases]

    # Year-by-year calculations
    for year in range(1, retirement_years + 1):
        df.loc[year, 'Age'] = age_retire + year

        # Adjust benefit for inflation
        df.loc[year, 'Annual_Benefit'] = df.loc[year-1, 'Annual_Benefit'] * (1 + inflation_rate)

        # Adjust budget for inflation and phase of retirement
        phase_factor = spending_adjustments[year-1] if year-1 < len(spending_adjustments) else spending_adjustments[-1]
        df.loc[year, 'Annual_Budget'] = df.loc[0, 'Annual_Budget'] * (1 + inflation_rate) ** year * phase_factor

        # Healthcare costs grow faster than inflation
        df.loc[year, 'Healthcare_Costs'] = df.loc[year-1, 'Healthcare_Costs'] * (1 + inflation_rate + 0.02)

        # Calculate withdrawal needed
        total_expenses = df.loc[year, 'Annual_Budget'] + df.loc[year, 'Healthcare_Costs']

        # If using Roth (tax-free withdrawals), no tax adjustment needed
        if account_type == 'Roth IRA/401k':
            withdrawal_needed = total_expenses - df.loc[year, 'Annual_Benefit']
        else:
            # For traditional accounts, adjust for taxes
            withdrawal_needed = (total_expenses - df.loc[year, 'Annual_Benefit']) / (1 - tax_rate)

        df.loc[year, 'Withdrawal'] = max(0, withdrawal_needed)

        # Calculate withdrawal rate
        df.loc[year, 'Withdrawal_Rate'] = df.loc[year, 'Withdrawal'] / df.loc[year-1, 'Portfolio'] if df.loc[year-1, 'Portfolio'] > 0 else 0

        # Update portfolio value
        starting_portfolio = df.loc[year-1, 'Portfolio']
        investment_returns = (starting_portfolio - df.loc[year, 'Withdrawal'] / 2) * expected_return
        df.loc[year, 'Portfolio'] = starting_portfolio - df.loc[year, 'Withdrawal'] + investment_returns
        df.loc[year, 'Portfolio'] = max(0, df.loc[year, 'Portfolio'])

        # Track if we've run out of money
        if df.loc[year, 'Portfolio'] <= 0 and year < retirement_years:
            df.loc[year:, 'Portfolio'] = 0
            break

    # Monte Carlo simulation for success probability
    num_simulations = 1000
    success_count = 0

    historical_returns = np.random.normal(expected_return, 0.12, (num_simulations, retirement_years))

    for sim in range(num_simulations):
        sim_portfolio = future_portfolio
        for year in range(retirement_years):
            if sim_portfolio <= 0:
                break

            annual_benefit = monthly_benefit_income * 12 * (1 + inflation_rate) ** (year + years_until_retirement)

            # Apply spending phase adjustment
            phase_factor = spending_adjustments[year] if year < len(spending_adjustments) else spending_adjustments[-1]
            annual_budget = initial_retirement_budget * (1 + inflation_rate) ** year * phase_factor

            # Healthcare with additional growth
            healthcare_cost = healthcare_costs * 12 * (1 + inflation_rate + 0.02) ** (year + years_until_retirement)

            total_expenses = annual_budget + healthcare_cost

            # Apply tax treatment based on account type
            if account_type == 'Roth IRA/401k':
                withdrawal = max(0, total_expenses - annual_benefit)
            else:
                withdrawal = max(0, (total_expenses - annual_benefit) / (1 - tax_rate))

            # Apply this year's random return
            sim_portfolio = sim_portfolio - withdrawal + (sim_portfolio - withdrawal/2) * historical_returns[sim, year]

        if sim_portfolio > 0:
            success_count += 1

    success_probability = success_count / num_simulations * 100

    # Output calculation results
    output = Output()
    with output:
        print(f"\n--- Enhanced Retirement Projection ---")
        print(f"Current Age: {age_current}, Retirement Age: {age_retire}, Life Expectancy: {life_expectancy}")
        print(f"Years Until Retirement: {years_until_retirement}, Years In Retirement: {retirement_years}")
        print(f"\nInitial Monthly Benefits: ${monthly_benefit_income:,.2f}")
        print(f"Current Portfolio: ${portfolio_total:,.2f}")
        print(f"Yearly Investment: ${yearly_investment:,.2f} for {years_contributing} years")
        print(f"Projected Portfolio at Retirement: ${future_portfolio:,.2f}")
        print(f"Current Monthly Budget: ${current_monthly_budget:,.2f}")
        print(f"Retirement Budget Ratio: {retirement_budget_ratio:.0%}")
        print(f"Initial Retirement Monthly Budget: ${initial_retirement_budget/12:,.2f}")
        print(f"Monthly Healthcare Costs at Retirement: ${df.loc[0, 'Healthcare_Costs']/12:,.2f}")
        print(f"\nExpected Annual Return: {expected_return:.2%}")
        print(f"Inflation Rate: {inflation_rate:.2%}")
        print(f"Effective Tax Rate: {tax_rate:.0%}")
        print(f"Retirement Spending Pattern: {retirement_phases}")

        if df['Portfolio'].iloc[-1] > 0:
            print(f"\nRESULT: Portfolio survives to end of life expectancy")
            print(f"Final Portfolio Value at Age {life_expectancy}: ${df['Portfolio'].iloc[-1]:,.2f}")
        else:
            portfolio_depletion_age = df.loc[df['Portfolio'] <= 0, 'Age'].min()
            print(f"\nRESULT: Portfolio depleted at age {portfolio_depletion_age:.0f}")

        print(f"\nMonte Carlo Success Probability: {success_probability:.1f}%")

        # Create visualization
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15))

        # Pre-retirement portfolio growth
        ax1.plot(pre_retirement_df['Age'], pre_retirement_df['Portfolio'], 'g-', linewidth=2)
        ax1.bar(pre_retirement_df['Age'], pre_retirement_df['Yearly_Contribution'], color='blue', alpha=0.3, label='Annual Contributions')
        ax1.set_title('Pre-Retirement Portfolio Growth')
        ax1.set_xlabel('Age')
        ax1.set_ylabel('Portfolio Value ($)')
        ax1.grid(True)
        ax1.legend()

        # Portfolio value over time during retirement
        ax2.plot(df['Age'], df['Portfolio'], 'b-', linewidth=2)
        ax2.set_title('Retirement Portfolio Value by Age')
        ax2.set_xlabel('Age')
        ax2.set_ylabel('Portfolio Value ($)')
        ax2.grid(True)

        # Income, expenses, and withdrawals
        ax3.bar(df['Age'], df['Annual_Benefit'], label='Benefit Income', color='green', alpha=0.6)
        ax3.bar(df['Age'], df['Withdrawal'], bottom=df['Annual_Benefit'], label='Portfolio Withdrawals', color='red', alpha=0.6)
        ax3.plot(df['Age'], df['Annual_Budget'], 'k--', label='Living Expenses', linewidth=2)
        ax3.plot(df['Age'], df['Healthcare_Costs'], 'r--', label='Healthcare Costs', linewidth=2)
        ax3.plot(df['Age'], df['Annual_Budget'] + df['Healthcare_Costs'], 'b-', label='Total Expenses', linewidth=2)
        ax3.set_title('Retirement Income and Expenses by Age')
        ax3.set_xlabel('Age')
        ax3.set_ylabel('Amount ($)')
        ax3.grid(True)
        ax3.legend()

        plt.tight_layout()
        plt.show()

        # Display the pre-retirement data table
        print("\nPre-Retirement Portfolio Growth:")
        display_cols_pre = ['Age', 'Portfolio', 'Yearly_Contribution']
        # Use pd.concat instead of append
        display(pd.concat([pre_retirement_df[display_cols_pre].head(), pre_retirement_df[display_cols_pre].tail()]))
    
        # Create visualization
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 20))  # Added ax4 for the new chart

        # Pre-retirement portfolio growth
        ax1.plot(pre_retirement_df['Age'], pre_retirement_df['Portfolio'], 'g-', linewidth=2)
        ax1.bar(pre_retirement_df['Age'], pre_retirement_df['Yearly_Contribution'], color='blue', alpha=0.3, label='Annual Contributions')
        ax1.set_title('Pre-Retirement Portfolio Growth')
        ax1.set_xlabel('Age')
        ax1.set_ylabel('Portfolio Value ($)')
        ax1.grid(True)
        ax1.legend()

        # Portfolio value over time during retirement
        ax2.plot(df['Age'], df['Portfolio'], 'b-', linewidth=2)
        ax2.set_title('Retirement Portfolio Value by Age')
        ax2.set_xlabel('Age')
        ax2.set_ylabel('Portfolio Value ($)')
        ax2.grid(True)

        # Income, expenses, and withdrawals
        ax3.bar(df['Age'], df['Annual_Benefit'], label='Benefit Income', color='green', alpha=0.6)
        ax3.bar(df['Age'], df['Withdrawal'], bottom=df['Annual_Benefit'], label='Portfolio Withdrawals', color='red', alpha=0.6)
        ax3.plot(df['Age'], df['Annual_Budget'], 'k--', label='Living Expenses', linewidth=2)
        ax3.plot(df['Age'], df['Healthcare_Costs'], 'r--', label='Healthcare Costs', linewidth=2)
        ax3.plot(df['Age'], df['Annual_Budget'] + df['Healthcare_Costs'], 'b-', label='Total Expenses', linewidth=2)
        ax3.set_title('Retirement Income and Expenses by Age')
        ax3.set_xlabel('Age')
        ax3.set_ylabel('Amount ($)')
        ax3.grid(True)
        ax3.legend()

        # New chart: Yearly budget, benefit income, and investment income
        ax4.plot(df['Age'], df['Annual_Budget'], label='Yearly Budget', color='blue', linewidth=2)
        ax4.plot(df['Age'], df['Annual_Benefit'], label='Benefit Income', color='green', linewidth=2)
        ax4.plot(df['Age'], df['Portfolio'] * expected_return, label='Investment Income', color='orange', linewidth=2)
        ax4.set_title('Yearly Budget, Benefit Income, and Investment Income')
        ax4.set_xlabel('Age')
        ax4.set_ylabel('Amount ($)')
        ax4.grid(True)
        ax4.legend()

        plt.tight_layout()
        plt.show()
    return output

# --- Interactive Widget with More Parameters ---
def create_retirement_planner():
    # Create output for results
    output = Output()

    # Build the interactive dashboard
    dashboard = interact(
        enhanced_retirement_calculator,
        age_current=IntSlider(min=25, max=70, step=1, value=45, description='Current Age'),
        age_retire=IntSlider(min=55, max=75, step=1, value=65, description='Retirement Age'),
        life_expectancy=IntSlider(min=70, max=100, step=1, value=90, description='Life Expectancy'),
        monthly_benefit_income=IntSlider(min=0, max=10000, step=100, value=2000, description='Benefits ($/mo)'),
        portfolio_total=IntSlider(min=0, max=5000000, step=10000, value=500000, description='Current Portfolio ($)'),
        yearly_investment=IntSlider(min=0, max=100000, step=1000, value=10000, description='Yearly Investment ($)'), # Added widget for yearly_investment
        years_contributing=IntSlider(min=0, max=40, step=1, value=20, description='Years Contributing'), # Added widget for years_contributing
        current_monthly_budget=IntSlider(min=1000, max=15000, step=100, value=5000, description='Current Budget ($/mo)'),
        inflation_rate=FloatSlider(min=0.01, max=0.05, step=0.001, value=0.025, description='Inflation Rate'),
        expected_return=FloatSlider(min=0.02, max=0.10, step=0.001, value=0.055, description='Expected Return'),
        account_type=Dropdown(
            options=['Roth IRA/401k', 'Traditional IRA/401k', 'Taxable Account'],
            value='Roth IRA/401k',
            description='Account Type'
        ),
        tax_rate=FloatSlider(min=0.0, max=0.4, step=0.01, value=0.15, description='Effective Tax Rate'),
        retirement_budget_ratio=FloatSlider(min=0.5, max=1.2, step=0.05, value=0.8, description='Budget Ratio'),
        retirement_phases=Dropdown(
            options=['Constant', 'Early Active', 'Late Increase'],
            value='Constant',
            description='Spending Pattern'
        ),
        healthcare_costs=IntSlider(min=200, max=2000, step=50, value=500, description='Healthcare ($/mo)')
    )

    return dashboard

    create_retirement_planner()

In [ ]:
create_retirement_planner()

In [1]:
from flask import Flask, render_template, request, jsonify
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use(‘Agg’)  # Use non-interactive backend
import matplotlib.pyplot as plt
import base64
import io
import json

app = Flask(**name**)

class EnhancedRetirementCalculator:
def **init**(self, age_current, age_retire, life_expectancy, monthly_benefit_income,
portfolio_total, yearly_investment, years_contributing, current_monthly_budget,
inflation_rate, expected_return, account_type, tax_rate, retirement_budget_ratio,
retirement_phases, healthcare_costs):

```
    self.age_current = age_current
    self.age_retire = age_retire
    self.life_expectancy = life_expectancy
    self.monthly_benefit_income = monthly_benefit_income
    self.portfolio_total = portfolio_total
    self.yearly_investment = yearly_investment
    self.years_contributing = years_contributing
    self.current_monthly_budget = current_monthly_budget
    self.inflation_rate = inflation_rate
    self.expected_return = expected_return
    self.account_type = account_type
    self.tax_rate = tax_rate
    self.retirement_budget_ratio = retirement_budget_ratio
    self.retirement_phases = retirement_phases
    self.healthcare_costs = healthcare_costs

def calculate(self):
    # Calculate years in retirement
    retirement_years = self.life_expectancy - self.age_retire
    years_until_retirement = self.age_retire - self.age_current
    years_contributing = min(self.years_contributing, years_until_retirement)

    # Create pre-retirement portfolio projection
    pre_retirement_df = pd.DataFrame(index=range(years_until_retirement + 1))
    pre_retirement_df.loc[0, 'Age'] = self.age_current
    pre_retirement_df.loc[0, 'Portfolio'] = self.portfolio_total
    pre_retirement_df.loc[0, 'Yearly_Contribution'] = self.yearly_investment if years_contributing > 0 else 0

    # Calculate portfolio growth during working years
    for year in range(1, years_until_retirement + 1):
        pre_retirement_df.loc[year, 'Age'] = self.age_current + year
        contribution = self.yearly_investment if year <= years_contributing else 0
        pre_retirement_df.loc[year, 'Yearly_Contribution'] = contribution
        
        previous_portfolio = pre_retirement_df.loc[year-1, 'Portfolio']
        investment_return = (previous_portfolio + contribution) * self.expected_return
        pre_retirement_df.loc[year, 'Portfolio'] = previous_portfolio + contribution + investment_return

    future_portfolio = pre_retirement_df.loc[years_until_retirement, 'Portfolio']

    # Create retirement dataframe
    df = pd.DataFrame(index=range(retirement_years + 1))
    df.loc[0, 'Age'] = self.age_retire
    df.loc[0, 'Portfolio'] = future_portfolio
    df.loc[0, 'Annual_Benefit'] = self.monthly_benefit_income * 12

    # Initial budget adjusted for inflation
    initial_retirement_budget = (self.current_monthly_budget * 12 * 
                               self.retirement_budget_ratio * 
                               (1 + self.inflation_rate) ** years_until_retirement)
    df.loc[0, 'Annual_Budget'] = initial_retirement_budget
    df.loc[0, 'Healthcare_Costs'] = (self.healthcare_costs * 12 * 
                                   (1 + self.inflation_rate) ** years_until_retirement)

    # Apply spending phases
    phase_adjustments = {
        'Constant': [1.0] * retirement_years,
        'Early Active': [1.2] * min(10, retirement_years) + [0.9] * max(0, retirement_years - 10),
        'Late Increase': [0.9] * min(20, retirement_years) + [1.3] * max(0, retirement_years - 20)
    }
    spending_adjustments = phase_adjustments[self.retirement_phases]

    # Year-by-year calculations
    for year in range(1, retirement_years + 1):
        df.loc[year, 'Age'] = self.age_retire + year
        df.loc[year, 'Annual_Benefit'] = df.loc[year-1, 'Annual_Benefit'] * (1 + self.inflation_rate)
        
        phase_factor = spending_adjustments[year-1] if year-1 < len(spending_adjustments) else spending_adjustments[-1]
        df.loc[year, 'Annual_Budget'] = df.loc[0, 'Annual_Budget'] * (1 + self.inflation_rate) ** year * phase_factor
        df.loc[year, 'Healthcare_Costs'] = df.loc[year-1, 'Healthcare_Costs'] * (1 + self.inflation_rate + 0.02)

        total_expenses = df.loc[year, 'Annual_Budget'] + df.loc[year, 'Healthcare_Costs']
        
        if self.account_type == 'Roth IRA/401k':
            withdrawal_needed = total_expenses - df.loc[year, 'Annual_Benefit']
        else:
            withdrawal_needed = (total_expenses - df.loc[year, 'Annual_Benefit']) / (1 - self.tax_rate)
        
        df.loc[year, 'Withdrawal'] = max(0, withdrawal_needed)
        df.loc[year, 'Withdrawal_Rate'] = (df.loc[year, 'Withdrawal'] / df.loc[year-1, 'Portfolio'] 
                                         if df.loc[year-1, 'Portfolio'] > 0 else 0)

        starting_portfolio = df.loc[year-1, 'Portfolio']
        investment_returns = (starting_portfolio - df.loc[year, 'Withdrawal'] / 2) * self.expected_return
        df.loc[year, 'Portfolio'] = starting_portfolio - df.loc[year, 'Withdrawal'] + investment_returns
        df.loc[year, 'Portfolio'] = max(0, df.loc[year, 'Portfolio'])

        if df.loc[year, 'Portfolio'] <= 0 and year < retirement_years:
            df.loc[year:, 'Portfolio'] = 0
            break

    # Monte Carlo simulation
    success_probability = self.monte_carlo_simulation(
        future_portfolio, retirement_years, years_until_retirement,
        initial_retirement_budget, spending_adjustments
    )

    return {
        'pre_retirement_df': pre_retirement_df,
        'retirement_df': df,
        'success_probability': success_probability,
        'future_portfolio': future_portfolio,
        'years_until_retirement': years_until_retirement,
        'retirement_years': retirement_years,
        'initial_retirement_budget': initial_retirement_budget
    }

def monte_carlo_simulation(self, future_portfolio, retirement_years, years_until_retirement,
                         initial_retirement_budget, spending_adjustments):
    num_simulations = 1000
    success_count = 0
    
    historical_returns = np.random.normal(self.expected_return, 0.12, (num_simulations, retirement_years))
    
    for sim in range(num_simulations):
        sim_portfolio = future_portfolio
        for year in range(retirement_years):
            if sim_portfolio <= 0:
                break
            
            annual_benefit = (self.monthly_benefit_income * 12 * 
                            (1 + self.inflation_rate) ** (year + years_until_retirement))
            
            phase_factor = spending_adjustments[year] if year < len(spending_adjustments) else spending_adjustments[-1]
            annual_budget = initial_retirement_budget * (1 + self.inflation_rate) ** year * phase_factor
            
            healthcare_cost = (self.healthcare_costs * 12 * 
                             (1 + self.inflation_rate + 0.02) ** (year + years_until_retirement))
            
            total_expenses = annual_budget + healthcare_cost
            
            if self.account_type == 'Roth IRA/401k':
                withdrawal = max(0, total_expenses - annual_benefit)
            else:
                withdrawal = max(0, (total_expenses - annual_benefit) / (1 - self.tax_rate))
            
            sim_portfolio = sim_portfolio - withdrawal + (sim_portfolio - withdrawal/2) * historical_returns[sim, year]
        
        if sim_portfolio > 0:
            success_count += 1
    
    return success_count / num_simulations * 100

def create_charts(self, results):
    """Create charts and return as base64 encoded strings"""
    pre_retirement_df = results['pre_retirement_df']
    df = results['retirement_df']
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Pre-retirement portfolio growth
    ax1.plot(pre_retirement_df['Age'], pre_retirement_df['Portfolio'], 'g-', linewidth=2)
    ax1.bar(pre_retirement_df['Age'], pre_retirement_df['Yearly_Contribution'], 
            color='blue', alpha=0.3, label='Annual Contributions')
    ax1.set_title('Pre-Retirement Portfolio Growth')
    ax1.set_xlabel('Age')
    ax1.set_ylabel('Portfolio Value ($)')
    ax1.grid(True)
    ax1.legend()
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
    
    # Portfolio value during retirement
    ax2.plot(df['Age'], df['Portfolio'], 'b-', linewidth=2)
    ax2.set_title('Retirement Portfolio Value by Age')
    ax2.set_xlabel('Age')
    ax2.set_ylabel('Portfolio Value ($)')
    ax2.grid(True)
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
    
    # Income and expenses
    ax3.bar(df['Age'], df['Annual_Benefit'], label='Benefit Income', color='green', alpha=0.6)
    ax3.bar(df['Age'], df['Withdrawal'], bottom=df['Annual_Benefit'], 
            label='Portfolio Withdrawals', color='red', alpha=0.6)
    ax3.plot(df['Age'], df['Annual_Budget'], 'k--', label='Living Expenses', linewidth=2)
    ax3.plot(df['Age'], df['Healthcare_Costs'], 'r--', label='Healthcare Costs', linewidth=2)
    ax3.set_title('Retirement Income and Expenses by Age')
    ax3.set_xlabel('Age')
    ax3.set_ylabel('Amount ($)')
    ax3.grid(True)
    ax3.legend()
    ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
    
    # Budget vs Income comparison
    ax4.plot(df['Age'], df['Annual_Budget'], label='Yearly Budget', color='blue', linewidth=2)
    ax4.plot(df['Age'], df['Annual_Benefit'], label='Benefit Income', color='green', linewidth=2)
    ax4.plot(df['Age'], df['Portfolio'] * self.expected_return, label='Investment Income', 
            color='orange', linewidth=2)
    ax4.set_title('Yearly Budget vs Income Sources')
    ax4.set_xlabel('Age')
    ax4.set_ylabel('Amount ($)')
    ax4.grid(True)
    ax4.legend()
    ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
    
    plt.tight_layout()
    
    # Convert plot to base64 string
    img = io.BytesIO()
    plt.savefig(img, format='png', dpi=150, bbox_inches='tight')
    img.seek(0)
    plot_url = base64.b64encode(img.getvalue()).decode()
    plt.close()
    
    return plot_url
```

@app.route(’/’)
def index():
return render_template(‘calculator.html’)

@app.route(’/calculate’, methods=[‘POST’])
def calculate():
try:
data = request.get_json()

```
    calculator = EnhancedRetirementCalculator(
        age_current=int(data['age_current']),
        age_retire=int(data['age_retire']),
        life_expectancy=int(data['life_expectancy']),
        monthly_benefit_income=float(data['monthly_benefit_income']),
        portfolio_total=float(data['portfolio_total']),
        yearly_investment=float(data['yearly_investment']),
        years_contributing=int(data['years_contributing']),
        current_monthly_budget=float(data['current_monthly_budget']),
        inflation_rate=float(data['inflation_rate']) / 100,
        expected_return=float(data['expected_return']) / 100,
        account_type=data['account_type'],
        tax_rate=float(data['tax_rate']) / 100,
        retirement_budget_ratio=float(data['retirement_budget_ratio']) / 100,
        retirement_phases=data['retirement_phases'],
        healthcare_costs=float(data['healthcare_costs'])
    )
    
    results = calculator.calculate()
    chart_data = calculator.create_charts(results)
    
    # Format results for display
    df = results['retirement_df']
    final_portfolio = df['Portfolio'].iloc[-1]
    portfolio_survives = final_portfolio > 0
    
    if not portfolio_survives:
        depletion_age = df.loc[df['Portfolio'] <= 0, 'Age'].min()
    else:
        depletion_age = None
    
    response_data = {
        'success': True,
        'results': {
            'future_portfolio': f"${results['future_portfolio']:,.2f}",
            'years_until_retirement': results['years_until_retirement'],
            'retirement_years': results['retirement_years'],
            'initial_monthly_budget': f"${results['initial_retirement_budget']/12:,.2f}",
            'portfolio_survives': portfolio_survives,
            'final_portfolio': f"${final_portfolio:,.2f}" if portfolio_survives else None,
            'depletion_age': int(depletion_age) if depletion_age else None,
            'success_probability': f"{results['success_probability']:.1f}%",
            'chart': chart_data
        }
    }
    
    return jsonify(response_data)

except Exception as e:
    return jsonify({'success': False, 'error': str(e)})
```

if **name** == ‘**main**’:
app.run(debug=True)

SyntaxError: invalid character '‘' (U+2018) (2288162963.py, line 5)